In [ ]:
# %%capture
! pip install --upgrade --force-reinstall git+https://github.com/huggingface/transformers

In [1]:
%cd MaskDetection/

/content/drive/MyDrive/MaskDetection


In [2]:
import torch
import torchvision.transforms as transforms
import os
import pandas as pd
from skimage import io
from torch.utils.data import (
    Dataset,
    DataLoader
)

from transformers import DeiTFeatureExtractor, DeiTForImageClassification

model_name_or_path = 'facebook/deit-base-distilled-patch16-224'
feature_extractor = DeiTFeatureExtractor.from_pretrained(model_name_or_path)

class FeatureExtractor(object):
    def __call__(self, image, target):
        sample = feature_extractor(image, return_tensors='pt')
        sample["labels"] = target
        return sample

class MaskDataset(Dataset):
    def __init__(self, csv_file, root_dir, transform=None):
        self.annotations = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.annotations)

    def __getitem__(self, index):
        img_path = os.path.join(self.root_dir, self.annotations.iloc[index, 0])
        image = io.imread(img_path)
        y_label = torch.tensor(int(self.annotations.iloc[index, 1]))
        
        data = self.transform(image,y_label)

        return data

train_ds = MaskDataset(
    csv_file="train.csv",
    root_dir="",
    # transform=transforms.ToTensor(),
    transform=FeatureExtractor(),
)

test_ds = MaskDataset(
    csv_file="test.csv",
    root_dir="",
    # transform=transforms.ToTensor(),
    transform=FeatureExtractor(),
)

Downloading:   0%|          | 0.00/287 [00:00<?, ?B/s]

In [3]:
train_ds.__getitem__(0)['pixel_values'].shape

torch.Size([1, 3, 224, 224])

In [ ]:
%pip install datasets

In [6]:
import numpy as np
from datasets import load_metric

metric = load_metric("accuracy")
def compute_metrics(p):
    return metric.compute(predictions=np.argmax(p.predictions, axis=1), references=p.label_ids)

Downloading:   0%|          | 0.00/1.41k [00:00<?, ?B/s]

In [7]:
labels = ["No Mask","Mask"]

model = DeiTForImageClassification.from_pretrained(
    model_name_or_path,
    num_labels=len(labels),
    ignore_mismatched_sizes=True,
    id2label={str(i): c for i, c in enumerate(labels)},
    label2id={c: str(i) for i, c in enumerate(labels)}
)

Downloading:   0%|          | 0.00/68.0k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/333M [00:00<?, ?B/s]

Some weights of the model checkpoint at facebook/deit-base-distilled-patch16-224 were not used when initializing DeiTForImageClassification: ['cls_classifier.weight', 'distillation_classifier.bias', 'distillation_classifier.weight', 'cls_classifier.bias']
- This IS expected if you are initializing DeiTForImageClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DeiTForImageClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DeiTForImageClassification were not initialized from the model checkpoint at facebook/deit-base-distilled-patch16-224 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream t

In [24]:
from transformers import TrainingArguments
from transformers import EarlyStoppingCallback

training_args = TrainingArguments(
  output_dir="./deit-base-distilled-mask-finetuned",
  per_device_train_batch_size=32,
  evaluation_strategy="steps",
  num_train_epochs=4,
  save_total_limit = 4, # Only last 4 models are saved. Older ones are deleted.
  fp16=True,
  save_steps=100,
  eval_steps=100,
  logging_steps=10,
  learning_rate=2e-4,
  remove_unused_columns=False,
  push_to_hub=False,
  report_to='tensorboard',
  load_best_model_at_end=True)

PyTorch: setting up devices


In [25]:
import torch

def collate_fn(batch):
    return {
        'pixel_values': torch.stack([x['pixel_values'][0] for x in batch]),
        'labels': torch.tensor([x['labels'] for x in batch])
    }

In [26]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=collate_fn,
    compute_metrics=compute_metrics,
    train_dataset=train_ds,
    eval_dataset=test_ds,
    tokenizer=feature_extractor,
    callbacks = [EarlyStoppingCallback(early_stopping_patience=3)],
)

Using amp half precision backend


In [27]:
train_results = trainer.train()
trainer.save_model()
trainer.log_metrics("train", train_results.metrics)
trainer.save_metrics("train", train_results.metrics)
trainer.save_state()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 14428
  Num Epochs = 4
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 1804


Step,Training Loss,Validation Loss,Accuracy
100,0.076500,0.051173,0.988877
200,0.053300,0.037400,0.990823
300,0.044200,0.039615,0.988598
400,0.035900,0.034941,0.988598
500,0.077700,0.042110,0.986930
600,0.005100,0.027107,0.992214


***** Running Evaluation *****
  Num examples = 3596
  Batch size = 8
Saving model checkpoint to ./deit-base-distilled-mask-finetuned/checkpoint-100
Configuration saved in ./deit-base-distilled-mask-finetuned/checkpoint-100/config.json
Model weights saved in ./deit-base-distilled-mask-finetuned/checkpoint-100/pytorch_model.bin
Feature extractor saved in ./deit-base-distilled-mask-finetuned/checkpoint-100/preprocessor_config.json
***** Running Evaluation *****
  Num examples = 3596
  Batch size = 8
Saving model checkpoint to ./deit-base-distilled-mask-finetuned/checkpoint-200
Configuration saved in ./deit-base-distilled-mask-finetuned/checkpoint-200/config.json
Model weights saved in ./deit-base-distilled-mask-finetuned/checkpoint-200/pytorch_model.bin
Feature extractor saved in ./deit-base-distilled-mask-finetuned/checkpoint-200/preprocessor_config.json
***** Running Evaluation *****
  Num examples = 3596
  Batch size = 8
Saving model checkpoint to ./deit-base-distilled-mask-finetuned/

Step,Training Loss,Validation Loss,Accuracy
100,0.076500,0.051173,0.988877
200,0.053300,0.037400,0.990823
300,0.044200,0.039615,0.988598
400,0.035900,0.034941,0.988598
500,0.077700,0.042110,0.986930
600,0.005100,0.027107,0.992214
700,0.011200,0.051631,0.984983
800,0.015200,0.058406,0.982481
900,0.033900,0.027991,0.990545


Saving model checkpoint to ./deit-base-distilled-mask-finetuned/checkpoint-700
Configuration saved in ./deit-base-distilled-mask-finetuned/checkpoint-700/config.json
Model weights saved in ./deit-base-distilled-mask-finetuned/checkpoint-700/pytorch_model.bin
Feature extractor saved in ./deit-base-distilled-mask-finetuned/checkpoint-700/preprocessor_config.json
Deleting older checkpoint [deit-base-distilled-mask-finetuned/checkpoint-300] due to args.save_total_limit
***** Running Evaluation *****
  Num examples = 3596
  Batch size = 8
Saving model checkpoint to ./deit-base-distilled-mask-finetuned/checkpoint-800
Configuration saved in ./deit-base-distilled-mask-finetuned/checkpoint-800/config.json
Model weights saved in ./deit-base-distilled-mask-finetuned/checkpoint-800/pytorch_model.bin
Feature extractor saved in ./deit-base-distilled-mask-finetuned/checkpoint-800/preprocessor_config.json
Deleting older checkpoint [deit-base-distilled-mask-finetuned/checkpoint-400] due to args.save_to

***** train metrics *****
  epoch                    =          2.0
  total_flos               = 2078245655GF
  train_loss               =       0.0438
  train_runtime            =   1:37:16.87
  train_samples_per_second =        9.887
  train_steps_per_second   =        0.309


In [28]:
metrics = trainer.evaluate(test_ds)
trainer.log_metrics("eval", metrics)
trainer.save_metrics("eval", metrics)

***** Running Evaluation *****
  Num examples = 3596
  Batch size = 8


***** eval metrics *****
  epoch                   =        2.0
  eval_accuracy           =     0.9922
  eval_loss               =     0.0271
  eval_runtime            = 0:03:17.36
  eval_samples_per_second =      18.22
  eval_steps_per_second   =       2.28
